In [28]:
import requests
from requests.adapters import HTTPAdapter, Retry
from azure.identity import ClientSecretCredential
import json
import os
from dotenv import load_dotenv
import yaml
load_dotenv()

tenant_id = os.getenv('tenant_id')
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
scope= 'offline_access https://analysis.windows.net/powerbi/api/.default'

In [29]:
redirect_uri = 'http://localhost/myapp/'
response_type = 'code'
response_mode = 'query'
# scope = ['https://api.fabric.microsoft.com/OneLake.ReadWrite.All']
state = '12345'

# Construct the URL
url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/authorize'

# Create the query parameters
params = {
    'client_id': client_id,
    'response_type': response_type,
    'redirect_uri': redirect_uri,
    'response_mode': response_mode,
    'scope': scope,
    'state': state
}

# Make the GET request
response = requests.get(url, params=params)

In [30]:
print(response.url)

https://login.microsoftonline.com/5cd64994-64c0-4c59-bca6-801527b1a508/oauth2/v2.0/authorize?client_id=80c581f3-02d7-4bbf-b371-8a61b80740ba&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2Fmyapp%2F&response_mode=query&scope=offline_access+https%3A%2F%2Fanalysis.windows.net%2Fpowerbi%2Fapi%2F.default&state=12345


In [31]:
code = '0.AcYAlEnWXMBkWUy8poAVJ7GlCPOBxYDXAr9Ls3GKYbgHQLrGAK0.AgABBAIAAADW6jl31mB3T7ugrWTT8pFeAwDs_wUA9P9_-MxiTqZ8q2GppY5X1XH7OTCJtt30pNqbZwV0FaBDvxJoiC__aI85Bf5_Qgev-CSDNhkMuA8td79EZB2cl2OL4s1ljjetrXg2TLArJMIzLkPTrQOe160BERMM65M4zJ6Br9N1SjUBuuELopq5OVQ1fPK2fvb-5klfDbTpvzTxe_I2QUFSLAoEHNnc_Ysru17Usg7MsJDbRFdenTTL39h7l53sw8mOZGena3ypcNfuK4zTKbuWjbufUKf0A6DrNAf1riRXoDYy81QtuGZNVm1zbA521dk7jHQiU5CerCLQAHlaRG44MdIkIHQugDyiWLgxKdyUni_Sc5h5tLPtZD6STjRVqFIUXibp0FXdkdACUJNzzNyQvucEcFESfXex4rlveAaIpiK7GUdXSVMgm9GhfafGLc9ct5vBMu1UzszZ_mDx5BVB2Dj08Qg6zyC60J4zxTOd_pYRXXpDqPJZqVrc0iLWYRUis0uji5By-RC3ia7NreKZ7k2wWIB3x9_cPWe1HDgy4SDiTcynI9rFmC7MMMlE0tCeQ6PVKMR4mtuBW7mDnEe6RIdmKdnAkJOAd-rvCs55f806YSDcNTqaIe8COmxvQ1Lms25yFg9oAWp47CMqIcI1gsLkXkzG7z9F7USldbdPqm8iiRbX17hVIflMFSbv0R-5siIYiHIoqjDut5oPXImft_ak8sToBS5f1j--k718hPCcQek0OfrjpshOObcLPA'

In [32]:
url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'

# Prepare the data for the request
data = {
    'client_id': client_id,
    'scope': scope,
    'code': code,
    'redirect_uri': 'http://localhost/myapp/',
    'grant_type': 'authorization_code',
    'client_secret': client_secret  # Only required for web apps
}

# Make the POST request
response = requests.post(url, data=data)

token_info = response.json()

In [33]:
token_info['access_token']

{'token_type': 'Bearer',
 'scope': 'https://analysis.windows.net/powerbi/api/OneLake.ReadWrite.All https://analysis.windows.net/powerbi/api/SQLEndpoint.Execute.All https://analysis.windows.net/powerbi/api/SQLEndpoint.Read.All https://analysis.windows.net/powerbi/api/SQLEndpoint.ReadWrite.All https://analysis.windows.net/powerbi/api/SQLEndpoint.Reshare.All https://analysis.windows.net/powerbi/api/Warehouse.Execute.All https://analysis.windows.net/powerbi/api/Warehouse.ExternalDataShare.All https://analysis.windows.net/powerbi/api/Warehouse.Read.All https://analysis.windows.net/powerbi/api/Warehouse.ReadWrite.All https://analysis.windows.net/powerbi/api/Warehouse.Reshare.All https://analysis.windows.net/powerbi/api/.default',
 'expires_in': 3676,
 'ext_expires_in': 3676,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwia

In [17]:
import struct
from itertools import chain, repeat

import pyodbc

In [18]:
sql_endpoint = "sre5mxgamrmuzpfgqakspmnfba-qof5ppqwvtoevjs2ev6jxgnd6y.datawarehouse.fabric.microsoft.com" # copy and paste the SQL endpoint from any of the Lakehouses or Warehouses in your Fabric Workspace
database = "lake1" # copy and paste the name of the Lakehouse or Warehouse you want to connect to

connection_string = f"Driver={{ODBC Driver 17 for SQL Server}};Server={sql_endpoint},1433;Database=f{database};Encrypt=Yes;TrustServerCertificate=No"

token_as_bytes = bytes(token_info['access_token'], "UTF-8") # Convert the token to a UTF-8 byte string
encoded_bytes = bytes(chain.from_iterable(zip(token_as_bytes, repeat(0)))) # Encode the bytes to a Windows byte string
token_bytes = struct.pack("<i", len(encoded_bytes)) + encoded_bytes # Package the token into a bytes object
attrs_before = {1256: token_bytes} 

In [21]:
encoded_bytes

b'e\x00y\x00J\x000\x00e\x00X\x00A\x00i\x00O\x00i\x00J\x00K\x00V\x001\x00Q\x00i\x00L\x00C\x00J\x00h\x00b\x00G\x00c\x00i\x00O\x00i\x00J\x00S\x00U\x00z\x00I\x001\x00N\x00i\x00I\x00s\x00I\x00n\x00g\x001\x00d\x00C\x00I\x006\x00I\x00j\x00N\x00Q\x00Y\x00U\x00s\x000\x00R\x00W\x00Z\x005\x00Q\x00k\x005\x00R\x00d\x00T\x00N\x00D\x00d\x00G\x00p\x00Z\x00c\x002\x00E\x00z\x00W\x00W\x001\x00o\x00U\x00T\x00V\x00F\x00M\x00C\x00I\x00s\x00I\x00m\x00t\x00p\x00Z\x00C\x00I\x006\x00I\x00j\x00N\x00Q\x00Y\x00U\x00s\x000\x00R\x00W\x00Z\x005\x00Q\x00k\x005\x00R\x00d\x00T\x00N\x00D\x00d\x00G\x00p\x00Z\x00c\x002\x00E\x00z\x00W\x00W\x001\x00o\x00U\x00T\x00V\x00F\x00M\x00C\x00J\x009\x00.\x00e\x00y\x00J\x00h\x00d\x00W\x00Q\x00i\x00O\x00i\x00J\x00o\x00d\x00H\x00R\x00w\x00c\x00z\x00o\x00v\x00L\x002\x00F\x00u\x00Y\x00W\x00x\x005\x00c\x002\x00l\x00z\x00L\x00n\x00d\x00p\x00b\x00m\x00R\x00v\x00d\x003\x00M\x00u\x00b\x00m\x00V\x000\x00L\x003\x00B\x00v\x00d\x002\x00V\x00y\x00Y\x00m\x00k\x00v\x00Y\x00X\x00B\x00p\x00I\x00i\x00w\x

In [23]:
import pyodbc

service_principal_id = f"{client_id}@{tenant_id}" # important to include your @fully qualified domain or tenant_id
service_principal_secret = client_secret

# Define your SQL Server details
server_name = "sre5mxgamrmuzpfgqakspmnfba-qof5ppqwvtoevjs2ev6jxgnd6y.datawarehouse.fabric.microsoft.com"
database_name = "lake1"
queryStr = 'SELECT * from dbo.test2'    

# Define the SQL Server ODBC connection string
conn_str = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server_name};"
    f"DATABASE={database_name};"
    f"UID={service_principal_id};"
    f"PWD={service_principal_secret};"
    f"Authentication=ActiveDirectoryServicePrincipal"
)

# Establish the connection
conn = pyodbc.connect(conn_str)

# Execute a query
cursor = conn.cursor()
cursor.execute(queryStr)
resultList = cursor.fetchall()
# resultColumns = columns = [column[0] for column in cursor.description]
# print(str([dict(zip(columns, row)) for row in resultList]))

InterfaceError: ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not login because the authentication failed. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not login because the authentication failed. (18456)')

In [24]:
pip install SQLAlchemy

     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.1 MB 217.9 kB/s eta 0:00:10
     ---------------------------------------- 0.0/2.1 MB 217.9 kB/s eta 0:00:10
      --------------------------------------- 0.0/2.1 MB 245.8 kB/s eta 0:00:09
     - -------------------------------------- 0.1/2.1 MB 297.7 kB/s eta 0:00:07
     -- ------------------------------------- 0.1/2.1 MB 516.7 kB/s eta 0:00:04
     ----- ---------------------------------- 0.3/2.1 MB 1.0 MB/s eta 0:00:02
     -------- ------------------------------- 0.5/2.1 MB 1.3 MB/s eta 0:00:02
     --------------- ------------------------ 0.8/2.1 MB 2.2 MB/s eta 0:00:01
     -------------------------- ------------- 1.4/2.1 MB 3.2 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/298.4 kB ? eta -:-


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import sqlalchemy as sa
from sqlalchemy import create_engine, MetaData, Table, select
import struct
from itertools import chain, repeat
import pyodbc
import urllib
from azure.identity import ClientSecretCredential

In [17]:
credential = ClientSecretCredential(tenant_id, client_id, client_secret)

In [20]:
# This will remain as is
resource_url = "https://analysis.windows.net/powerbi/api/.default"

# Get the token
token_object = credential.get_token(resource_url)

In [21]:
sql_endpoint  = "sre5mxgamrmuzpfgqakspmnfba-qof5ppqwvtoevjs2ev6jxgnd6y.datawarehouse.fabric.microsoft.com"
database  = "lake1"
queryStr = 'SELECT * from dbo.test2'    

In [22]:
connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint}"

In [24]:
token_object.token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNWNkNjQ5OTQtNjRjMC00YzU5LWJjYTYtODAxNTI3YjFhNTA4LyIsImlhdCI6MTcyOTg1MDI2MSwibmJmIjoxNzI5ODUwMjYxLCJleHAiOjE3Mjk4NTQxNjEsImFpbyI6ImsyQmdZTGpxdVdQU204eStCWXRGVTVmNkNSdHZCQUE9IiwiYXBwaWQiOiI4MGM1ODFmMy0wMmQ3LTRiYmYtYjM3MS04YTYxYjgwNzQwYmEiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81Y2Q2NDk5NC02NGMwLTRjNTktYmNhNi04MDE1MjdiMWE1MDgvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI0MDIwMTJkYS1mMTk1LTQ0MWItODBjMi01OGMwMWIwMDlkM2MiLCJyaCI6IjAuQWNZQWxFbldYTUJrV1V5OHBvQVZKN0dsQ0FrQUFBQUFBQUFBd0FBQUFBQUFBQURHQUFBLiIsInN1YiI6IjQwMjAxMmRhLWYxOTUtNDQxYi04MGMyLTU4YzAxYjAwOWQzYyIsInRpZCI6IjVjZDY0OTk0LTY0YzAtNGM1OS1iY2E2LTgwMTUyN2IxYTUwOCIsInV0aSI6ImNQRmpnS1Q0Q0VHOFdIdEdJLU9aQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfaWRyZWwiOiIyOCA3In0.UYD0Q027sdKWq2-8RyDX-H-E32juKc7aIGEiMvq0fa

In [37]:
token = token_info['access_token'].encode("UTF-16-LE")

In [38]:
token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
SQL_COPT_SS_ACCESS_TOKEN = 1256
engine = sa.create_engine("mssql+pyodbc://", creator=lambda: pyodbc.connect(connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}))

In [38]:
connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Database=f{database};Encrypt=Yes;TrustServerCertificate=No"
params = urllib.parse.quote(connection_string)

In [32]:
# Retrieve an access token valid to connect to SQL databases
token_as_bytes = bytes(token_object.token, "UTF-8") # Convert the token to a UTF-8 byte string
encoded_bytes = bytes(chain.from_iterable(zip(token_as_bytes, repeat(0)))) # Encode the bytes to a Windows byte string
token_bytes = struct.pack("<i", len(encoded_bytes)) + encoded_bytes # Package the token into a bytes object
attrs_before = {1256: token_bytes}  # Attribute pointing to SQL_COPT_SS_ACCESS_TOKEN to pass access token to the driver

In [36]:
# build the connection
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={0}".format(params), connect_args={'attrs_before': attrs_before})

NameError: name 'attrs_before' is not defined

In [36]:
import pandas as pd

In [39]:
sql_df = pd.read_sql( 
    "SELECT * FROM sys.objects", 
    con=engine 
) 

InterfaceError: (pyodbc.InterfaceError) ('28000', '[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed due to invalid authentication methods. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed due to invalid authentication methods. (18456)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [2]:
import pandas as pd
import sqlalchemy as sa
import urllib
import struct
import requests
import json

In [5]:
def get_access_token():
    """It will create a access token to access the mail apis"""
    app_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
    directory_id = os.getenv('tenant_id')
    token_url = "https://login.microsoftonline.com/"+directory_id+"/oauth2/token"
    token_data = {
    "grant_type": "password",
    "client_id": app_id,
    "client_secret": client_secret,
    "resource": "https://storage.azure.com/",
    "scope":"https://graph.microsoft.com",
    "username": os.getenv('username'), 
    "password": os.getenv('password'),
    }
    token_headers={
      "Content-Type":"application/x-www-form-urlencoded"
    }
    print(token_url)
    token_response = requests.post(token_url,data=token_data,headers=token_headers)
    token_response_dict = json.loads(token_response.text)
    error_exists = checkKey(token_response_dict,"error")
    #print(token_response.text)
    if error_exists == True :
      raise Exception("Error in getting in access token") 
    else:
      token = token_response_dict.get("access_token")
      refresh_token = token_response_dict.get("refresh_token")
      if token == None :
        print("Unable to get access token")
        print(str(token_response_dict))
        raise Exception("Error in getting in access token")
      elif refresh_token == None :
        print("Unable to get refresh token")
        print(str(token_response_dict))
        raise Exception("Error in getting in refresh token")
      else:
        return token

def checkKey(dict, key):
    """It will check the given key exists in the response dictionary and return True or False"""
    if key in dict.keys():
        # print(dict[key])
        return True
    else:
        return False
def checkKey(dict, key):
    """It will check the given key exists in the response dictionary and return True or False"""
    if key in dict.keys():
        # print(dict[key])
        return True
    else:
        return False

In [13]:
app_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
directory_id = '5cd64994-64c0-4c59-bca6-801527b1a508'
token_url = "https://login.microsoftonline.com/"+directory_id+"/oauth2/token"
token_data = {
"grant_type": "password",
"client_id": app_id,
"client_secret": client_secret,
"resource": "https://storage.azure.com/",
"scope":"https://graph.microsoft.com",
"username": os.getenv('user_name'), 
"password": os.getenv('password'),
}
token_headers={
    "Content-Type":"application/x-www-form-urlencoded"
}
print(token_url)
token_response = requests.post(token_url,data=token_data,headers=token_headers)

https://login.microsoftonline.com/5cd64994-64c0-4c59-bca6-801527b1a508/oauth2/token


In [15]:
token_response.text

'{"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access \'e406a681-f3d4-42a8-90b6-c2b029497af1\'. Trace ID: 48d9ea91-0224-45a4-a746-82a04a549100 Correlation ID: 63eaacf5-f0f8-4412-9c17-b6a61168a005 Timestamp: 2024-10-25 09:35:50Z","error_codes":[50076],"timestamp":"2024-10-25 09:35:50Z","trace_id":"48d9ea91-0224-45a4-a746-82a04a549100","correlation_id":"63eaacf5-f0f8-4412-9c17-b6a61168a005","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'